In [ ]:
import cv2
import numpy as np
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from keras.utils.np_utils import to_categorical
from keras.datasets import mnist

(x_train,y_train) , (x_test,y_test) = mnist.load_data()
x_train = x_train/x_train.max()
x_test = x_test/x_test.max()
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

model = Sequential()
model.add(Conv2D(input_shape = (28,28,1),kernel_size = (4,4),filters = 32,activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(Dense(10,activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

y_train_cat = to_categorical(y_train,10)
y_test_cat = to_categorical(y_test,10)

model.fit(x_train,y_train_cat)

threshold = 0.75
capture = cv2.VideoCapture(0)
while True :
    ret , actual_frame = capture.read()
    frame = np.asarray(actual_frame)
    frame = cv2.resize(frame,(28,28))
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    frame = frame/255
    frame = frame.reshape(1,28,28,1)
    number  = model.predict_classes(frame)
    proba = model.predict_proba(frame)
    result_str = ("Number - {}".format(number))
    result_str1 = ("Probability% - {}".format(int(100*proba.max())))
    proba = model.predict_proba(frame)
    if proba.max() > threshold :
        cv2.putText(actual_frame, str(result_str) , (0,50), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)
        cv2.putText(actual_frame, str(result_str1) , (0,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)
    cv2.imshow("x",actual_frame)
    k = cv2.waitKey(5) 
    if k == ord('q') :
        break

capture.release()
cv2.destroyAllWindows()